# Python Resources

## Information: Course

There is a software carpentry style course available on James Grant's webpage, available here:

[Python course](http://people.bath.ac.uk/rjg20/training/intro-python/)